In [1]:
from spectroscopy import LeafSampleReader, DataCleaner, DataReducer, TargetScaler, BaselineCorrector, Metrics, PerformancePlotter, PLSEstimator, RegressionModels
from spectroscopy.src.utility_functions import get_working_directory, train_test_split, load_config, create_timestamp_filename, get_log_level
from tabulate import tabulate
import logging
import os
import pandas as pd

In [2]:
working_directory_path = get_working_directory()
leaf_samples_folder_path = (f"{working_directory_path}/data/leaf_samples")

In [3]:
leaf_sample_reader = LeafSampleReader(leaf_samples_folder_path)

leaf_df = leaf_sample_reader.read_selected_csvs(seasons=[1,2,3,4], leaf_states=["dried"])

leaf_df = DataCleaner.enforce_data_types(leaf_df)

leaf_df = DataCleaner.drop_null_data(leaf_df, row_threshold=0.5, target_col_threshold=0.5, feature_col_threshold=0.5)

leaf_df = DataCleaner.impute_data(leaf_df, target_method="knn", feature_method="neighbour_avg")

In [7]:
leaf_df

,sample_id,N,P,Ca,K,Mn,Mg,B,S,Na,...,2496nm,2496.5nm,2497nm,2497.5nm,2498nm,2498.5nm,2499nm,2499.5nm,leaf_state,season
0,S02.01,5.03,0.37,1.65,9.57,62.0,0.48,24.40,0.20,0.02,...,0.275998,0.276023,0.276055,0.276095,0.276143,0.276199,0.276263,0.276337,DRIED,1
1,S02.02,5.40,0.36,2.22,9.57,66.0,0.62,20.00,0.20,0.02,...,0.283747,0.283762,0.283784,0.283813,0.283848,0.283892,0.283943,0.284005,DRIED,1
2,S02.03,5.27,0.37,1.91,8.92,39.0,0.53,24.40,0.20,0.02,...,0.265128,0.265130,0.265138,0.265151,0.265171,0.265196,0.265228,0.265270,DRIED,1
3,S02.04,4.89,0.24,1.05,8.80,416.0,0.39,42.00,0.23,0.02,...,0.325594,0.325591,0.325593,0.325602,0.325618,0.325641,0.325672,0.325715,DRIED,1
4,S02.05,5.14,0.28,1.21,9.14,445.0,0.30,41.00,0.21,0.02,...,0.337213,0.337217,0.337227,0.337241,0.337261,0.337287,0.337319,0.337362,DRIED,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,Hughes Fumi.C.07.25.23.d,4.47,0.43,0.81,8.92,303.0,0.35,20.97,0.23,0.01,...,0.280921,0.280984,0.281054,0.281131,0.281216,0.281308,0.281410,0.281523,DRIED,4
381,Hughes Fumi.C.08.08.08.d,4.46,0.19,1.10,6.82,746.0,0.53,17.92,0.14,0.01,...,0.251410,0.251538,0.251671,0.251809,0.251952,0.252102,0.252257,0.252421,DRIED,4
382,Hughes Fumi.C.45162.d,2.14,0.15,1.05,7.65,706.0,0.58,23.01,0.14,0.01,...,0.293060,0.293154,0.293253,0.293358,0.293470,0.293590,0.293718,0.293858,DRIED,4
383,Hughes Fumi.C.45175.d,1.36,0.10,0.80,6.57,269.0,0.23,19.83,0.13,0.01,...,0.309903,0.310021,0.310145,0.310274,0.310411,0.310554,0.310703,0.310862,DRIED,4


In [8]:
training_df, testing_df = train_test_split(leaf_df, method="stratified")

In [18]:
training_df[training_df["season"]==4].shape

(83, 4216)

In [19]:
training_df, _ = DataCleaner.remove_outliers(training_df, method="both", feature_outlier_threshold= 95)

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 29 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC1     31 non-null     float64
 1   PC2     31 non-null     float64
dtypes: float64(2)
memory usage: 744.0 bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, 62 to 115
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC1     115 non-null    float64
 1   PC2     115 non-null    float64
dtypes: float64(2)
memory usage: 2.7 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 79 entries, 203 to 205
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC1     79 non-null     float64
 1   PC2     79 non-null     float64
dtypes: float64(2)
memory usage: 1.9 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 83 entries, 327 to 311
Data columns (total 2 columns):
 #   Column  Non-Nu

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values